# Here I want to merge together the data from different sources
# TODO prio
### TODO Here i reduce all the rnaseq data! Check if thats right. Its already bit strange that there are that many duplicate subjects within the data
## find algorithm to make microarray and RNAseq data more comparable (OR. only use one of the techniques but increase data amount)
## merge COVID dataset from wimmers
## create gamlss models for the merged datasets

# TODO less
## impute mean value to nans
## calc percentage of nan per gene. 
## drop biomarkers for values with Nan


In [ ]:

%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
import seaborn as sns
from utils.helper import manhattanplot, violinplot_overall, scatter_plot, get_negative_values, METADATA_COLS


## Step 1 10k immu

In [ ]:
whole_blood_common = pd.read_csv('whole_blood/whole_blood_combi_common_columns.csv', index_col=0)
print(get_negative_values(whole_blood_common))
whole_blood_common.describe()

In [ ]:
pbmc_common = pd.read_csv('pbmc/combi_df_pbmc_ordered_common_cols.csv', index_col=0)
print(pbmc_common.shape)
print(get_negative_values(pbmc_common))
pbmc_common.describe()

In [ ]:
combi_df_all_reduced['METHOD'].unique()

In [ ]:
common_columns = whole_blood_common.columns.intersection(pbmc_common.columns)

combi_df_all = pd.concat([whole_blood_common, pbmc_common], ignore_index=True)
print(combi_df_all.shape)

### Remove possible duplicated subjects and entries with Nan 
#combi_df_all_reduced = combi_df_all.drop_duplicates()
combi_df_all_reduced = combi_df_all.drop_duplicates(subset=['subject_accession'], keep='first')
print(combi_df_all_reduced.shape)
###TODO Here i reduce all the rnaseq data! Check if thats right. Its already bit strange that there are that many duplicate subjects within the data

## drop entries that dont fullfill those min metadata
essential_columns = ['age', 'gender', 'subject_accession']
combi_df_all_reduced = combi_df_all_reduced.dropna(subset=essential_columns, axis=0, how='any')
print(combi_df_all_reduced.shape)



In [ ]:
manhattanplot(combi_df_all[combi_df_all['TYPE']=='pbmc'], start_col=9)


In [ ]:
combi_df_all_reduced.to_csv('10k_immu/combi_10k_all.csv')

In [ ]:
no_meta_cols = list(set(common_columns) - set(METADATA_COLS))
print(len(no_meta_cols))
new_order_cols = METADATA_COLS + no_meta_cols
print(len(new_order_cols))

In [ ]:
combi_df_common = combi_df_all_reduced[new_order_cols]
print(combi_df_common.shape)
combi_df_common.to_csv('10k_immu/combi_10k_common.csv')

In [ ]:
# WHAT ????? negative AND max values that are waaaay too off .... 
# max vals should be MAX 20 (as i log 2 transformed the data)
# no negative min vals 
combi_df_common.describe()

In [ ]:
combi_df_common.head()

In [ ]:
violinplot_overall(combi_df_common, x_topic='gender', y_topic='age', hue_split='gender', bin_name='10k_immu')

In [ ]:
manhattanplot(combi_df_common, start_col=9, save_name='10k_immu/combi_10k_common')
### The plot looks okay, because the median per gene is probably corrected by the increased number of subjects

In [ ]:
combi_df_common['METHOD'].unique()

In [ ]:
manhattanplot(combi_df_common[combi_df_common['METHOD']=='rnaseq'], start_col=9)


In [ ]:
manhattanplot(combi_df_common[combi_df_common['METHOD']=='array'], start_col=9)


In [ ]:
manhattanplot(combi_df_common[combi_df_common['TYPE']=='whole_blood'], start_col=9)


In [ ]:
print(combi_df_common[combi_df_common['TYPE']=='pbmc'].shape)


In [ ]:
manhattanplot(combi_df_common[combi_df_common['TYPE']=='pbmc'], start_col=9)


In [ ]:
def scatter_plot(df, x_ID='DUSP22', y_ID='age', hue_ID='study_accession'):
    plot = sns.jointplot(x=x_ID, y=y_ID, data=df, kind='scatter', hue=hue_ID)
    
    plot.ax_joint.legend_.remove()

    #plot.ax_joint.scatter(df[x_ID], df[y_ID], c=df['study_accession'])
    plt.xlabel(f'{x_ID}')
    plt.ylabel(f'{y_ID}')
   # plt.title(f'{x_ID} - {y_ID}')
    plt.show()
    
scatter_plot(combi_df_common)